## 03. Практический кейс. Исследовательский анализ данных.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)

In [66]:
# open data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

### Посмотрим на признаки

In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119728 entries, 0 to 119727
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             119728 non-null  object 
 1   brand                119728 non-null  object 
 2   color                119728 non-null  object 
 3   engineDisplacement   119728 non-null  float64
 4   enginePower          119728 non-null  float64
 5   fuelType             119728 non-null  object 
 6   mileage              119728 non-null  int64  
 7   modelName            119728 non-null  object 
 8   years                119728 non-null  int64  
 9   vehicleTransmission  119728 non-null  object 
 10  vendor               119728 non-null  object 
 11  Владельцы            119728 non-null  object 
 12  ПТС                  119728 non-null  object 
 13  Привод               119728 non-null  object 
 14  Руль                 119728 non-null  object 
 15  price            

In [58]:
train_df.head(5)

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelName,years,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,price,mileagePerYear
0,лифтбек,SKODA,белый,2.0,220.0,бензин,210000,OCTAVIA_RS,7,роботизированная,EUROPEAN,2 владельца,Оригинал,передний,Левый,1075000.0,26250.0
1,лифтбек,SKODA,синий,2.0,200.0,бензин,221000,OCTAVIA_RS,9,роботизированная,EUROPEAN,2 владельца,Оригинал,передний,Левый,872000.0,22100.0
2,лифтбек,SKODA,серый,2.0,200.0,бензин,86236,OCTAVIA_RS,9,роботизированная,EUROPEAN,2 владельца,Оригинал,передний,Левый,650000.0,8624.0
3,лифтбек,SKODA,чёрный,2.0,200.0,бензин,150000,OCTAVIA_RS,9,роботизированная,EUROPEAN,3 или более,Дубликат,передний,Левый,725000.0,15000.0
4,лифтбек,SKODA,серый,2.0,200.0,бензин,136505,OCTAVIA_RS,10,роботизированная,EUROPEAN,2 владельца,Оригинал,передний,Левый,740000.0,12410.0


#### Переименуем model_name

In [6]:
train_df = train_df.rename(columns={'model_name': 'modelName'})
test_df = test_df.rename(columns={'model_name': 'modelName'})

#### Поменяем productionDate на years

In [50]:
def convert_date_to_years(df):
    this_year = datetime.now().year
    years_passed = this_year - df['productionDate']
    
    # replace with years
    df.loc[:, 'productionDate'] = years_passed
    # rename
    df.rename(columns={'productionDate': 'years'}, inplace=True)

In [ ]:
convert_date_to_years(train_df)
convert_date_to_years(test_df)

#### Уберем \xa0 в Владельцы тестового датасета

In [63]:
test_df.loc[:, 'Владельцы'] = test_df['Владельцы'].apply(lambda x: x.replace('\xa0', ' '))

#### Удаляем авто моделей, которых нет в тестовом датасете

In [78]:
test_models = test_df['modelName'].unique()
train_models = train_df['modelName'].unique()

train_df = train_df[train_df['modelName'].isin(test_models)]

### Добавим некоторые признаки

#### Пробег за год

In [44]:
def add_mileage_per_year(df):
    years_passed = df['years'] + 1

    mileage_per_year = df['mileage'] / years_passed
    mileage_per_year[mileage_per_year == np.inf] = 0 # fix infinity values

    df.loc[:, 'mileagePerYear'] = np.round(mileage_per_year)

In [45]:
add_mileage_per_year(train_df)
add_mileage_per_year(test_df)

In [79]:
# save both dataframes
train_df.to_csv('data/train.csv', index=None)
test_df.to_csv('data/test.csv', index=None)

### Изучим корреляцию между признаками

In [7]:
# correlation of numerical features with price
train_df.corr()['price']

price                 1.000000
engineDisplacement    0.391119
enginePower           0.552892
mileage              -0.468470
productionDate        0.519956
Name: price, dtype: float64

In [13]:
# same as above, but more visual
corr = train_df[['enginePower', 'mileage', 'modelDate', 'numberOfDoors', 'productionDate', 'price']] \
                .dropna(axis=1) \
                .corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "1em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,enginePower,mileage,modelDate,numberOfDoors,productionDate,price
enginePower,1.00,-0.19,0.23,0.02,0.21,0.54
mileage,-0.19,1.00,-0.76,-0.15,-0.78,-0.50
modelDate,0.23,-0.76,1.00,0.25,0.98,0.48
numberOfDoors,0.02,-0.15,0.25,1.00,0.26,0.09
productionDate,0.21,-0.78,0.98,0.26,1.00,0.48
price,0.54,-0.50,0.48,0.09,0.48,1.00
